In [1]:
#import dependencies
import requests
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gmaps
import os 
import pprint
import urllib

#import yelp_key
from config import yelp_key 

In [2]:
philadelphia_zips_df = pd.read_csv("../Resources/philadelphia_zipcodes.csv")
philadelphia_zips_df.drop(philadelphia_zips_df.tail(1).index,inplace=True)
zips = [zips for zips in philadelphia_zips_df['Philadelphia'] ]
restaurant_ethnicities_df = pd.read_csv("../Resources/rest_enthinicity.csv")
ethnicities = [ethnicity for ethnicity in restaurant_ethnicities_df['ethnicity'] ]
ethnicities[ethnicities.index('indian')] = 'indpak'
ethnicities[ethnicities.index('brazillian')] = 'brazilian'
ethnicities[ethnicities.index('new american')] = 'newamerican'
ethnicities[ethnicities.index('soul food')] = 'soulfood'
ethnicities

['chinese',
 'french',
 'greek',
 'indpak',
 'italian',
 'japanese',
 'brazilian',
 'mexican',
 'newamerican',
 'soulfood',
 'thai']

In [3]:
responses = []
API_HOST = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'Bearer %s' % yelp_key}

for ethnicity in ethnicities:     
    params={'term':'restaurants','location':'Philadelphia', 'categories':f'{ethnicity}', 'offset':0}
    response = requests.get(API_HOST,headers=headers, params=params).json()
    response['ethnicity'] = f'{ethnicity}'
    responses.append(response)
    if response['total'] > 50:
        offset = 50
        while offset < response['total']:
            params={'term':'restaurants','location':'Philadelphia', 'categories':f'{ethnicity}', 'offset': offset}
            response = requests.get(API_HOST,headers=headers, params=params).json()
            response['ethnicity'] = f'{ethnicity}'
            responses.append(response)
            offset = offset + 50     

In [4]:
ethnicity_response_list = [responses[i]['ethnicity'] for i in range(len(responses))]
ethnicity_response_list

['chinese',
 'chinese',
 'chinese',
 'chinese',
 'chinese',
 'chinese',
 'chinese',
 'chinese',
 'chinese',
 'french',
 'greek',
 'indpak',
 'indpak',
 'italian',
 'italian',
 'italian',
 'italian',
 'italian',
 'italian',
 'japanese',
 'japanese',
 'japanese',
 'brazilian',
 'mexican',
 'mexican',
 'mexican',
 'mexican',
 'mexican',
 'newamerican',
 'newamerican',
 'newamerican',
 'newamerican',
 'newamerican',
 'newamerican',
 'soulfood',
 'soulfood',
 'thai',
 'thai']

In [5]:
name = []
city = []
latitude = []
longitude = []
zipcode = []
ethnicity = []
price = []
rating = []

for i in range(len(responses)):
    for j in range(len(responses[i]['businesses'])):
        name.append(responses[i]["businesses"][j]["name"])
        city.append("Philadelphia")
        latitude.append(responses[i]["businesses"][j]["coordinates"]["latitude"])
        longitude.append(responses[i]["businesses"][j]["coordinates"]["longitude"])
        zipcode.append(responses[i]["businesses"][j]["location"]["zip_code"])
        ethnicity.append((responses[i]['ethnicity']))
        try:
            price.append(responses[i]["businesses"][j]["price"])
        except KeyError:
            price.append("N/A")
        rating.append(responses[i]["businesses"][j]["rating"])

In [6]:
philly_rest_df = pd.DataFrame({
    "name": name,
    "city": city,
    "latitude": latitude,
    "longitude": longitude,
    "zipcode": zipcode,
    "ethnicity": ethnicity,
    "price": price,
    "rating": rating
})

philly_rest_df

,name,city,latitude,longitude,zipcode,ethnicity,price,rating
0,Nan Zhou Hand Drawn Noodle House,Philadelphia,39.955401,-75.156901,19107,chinese,$,4.0
1,Dim Sum Garden,Philadelphia,39.955399,-75.156727,19107,chinese,$,4.0
2,Chubby Cattle,Philadelphia,39.954980,-75.156220,19107,chinese,N/A,4.5
3,Dim Sum House,Philadelphia,39.951885,-75.172466,19103,chinese,$$,4.0
4,Dan Dan,Philadelphia,39.950249,-75.167595,19102,chinese,$$,4.0
...,...,...,...,...,...,...,...,...
714,Siamese Princess,Philadelphia,40.006996,-75.289871,19003,thai,$$,4.0
715,Mikado Thai Pepper Restaurant,Philadelphia,40.006550,-75.288880,19003,thai,$$,3.5
716,Aya Asian Fusion,Philadelphia,40.036950,-75.039910,19136,thai,$$,3.5
717,Aziatisch,Philadelphia,40.007524,-75.285984,19003,thai,$$,4.0


In [8]:
philly_rest_df.to_csv("../Output/philadelphia_restaurants.csv", header = True, index = False)